En este archivo vamos a entrenar una red neuronal para predecir el resultado de los partidos de la Champions bsándonos en datos de los último 7 años.

Para empezar primero cargamos las librerias necesarias.

In [337]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import to_categorical
from keras.layers import Dropout

Ahora vamos a empezar el entrenamiento para predecir resultados, pero antes de ello vamos a transformar las columnas string de equipos en numéricas para poder evaluarlas y analizarlas durante el procesamiento de la red neuronal.

In [338]:
# Cargar los datos del ranking UEFA
uefa_ranking = pd.read_csv("data/UEFA_Ranking.csv")

# Crear un diccionario para mapear nombres de equipos a números
team_mapping = {team: rank for rank, team in zip(uefa_ranking['Position'], uefa_ranking['Club'])}

# Cargar los datos de los partidos de la Champions League 2017
champions_data = pd.read_csv("data/champions-league-2017.csv")

# Reemplazar nombres de equipos por números en las columnas correspondientes
champions_data['Home Team'] = champions_data['Home Team'].map(team_mapping)
champions_data['Away Team'] = champions_data['Away Team'].map(team_mapping)
champions_data['Winner'] = champions_data['Winner'].map(team_mapping)

Vamos a comprobar que todos los datos están bien.

In [339]:
# Ver las primeras filas de los datos
print(champions_data.head())

   Match Number Round Number  Home Team  Away Team Result  Winner  \
0             1            1         20         61  1 - 2      61   
1             2            1         11         26  3 - 0      11   
2            13            1          2         52  3 - 0       2   
3            14            1         50          8  0 - 5       8   
4            25            1          9         66  6 - 0       9   

   Home Team Goals  Away Team Goals  Match Goals  
0                1                2            3  
1                3                0            3  
2                3                0            3  
3                0                5            5  
4                6                0            6  


In [340]:
# analizar los datos
print(champions_data.describe())

       Match Number   Home Team   Away Team     Winner  Home Team Goals  \
count    125.000000  125.000000  125.000000  125.00000       125.000000   
mean      63.000000   25.288000   25.320000   17.24800         1.784000   
std       36.228442   28.268247   28.242784   22.06722         1.532361   
min        1.000000    1.000000    1.000000    1.00000         0.000000   
25%       32.000000    6.000000    6.000000    4.00000         1.000000   
50%       63.000000   14.000000   14.000000   10.00000         1.000000   
75%       94.000000   36.000000   36.000000   23.00000         3.000000   
max      125.000000  119.000000  119.000000  119.00000         7.000000   

       Away Team Goals  Match Goals  
count       125.000000   125.000000  
mean          1.424000     3.208000  
std           1.398525     1.622901  
min           0.000000     0.000000  
25%           0.000000     2.000000  
50%           1.000000     3.000000  
75%           2.000000     4.000000  
max           7.0000

In [341]:
# ver los datos nulos
print(champions_data.isnull().sum())

Match Number       0
Round Number       0
Home Team          0
Away Team          0
Result             0
Winner             0
Home Team Goals    0
Away Team Goals    0
Match Goals        0
dtype: int64


Como podemos ver está todo bien. Para más información con respecto al CSV que estamos tratando dirigirse a graficas.ipynb

Ahora vamos a comenzar el proceso de entrenamiento de la red neuronal

In [342]:
'''# Preprocesamiento de datos
X = champions_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals']].values
y = champions_data['Winner'].values'''

# Preprocesamiento de datos
X = champions_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals']].values
y = champions_data['Winner'].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Goals', 'Away Team Goals']

X_categorical = champions_data[categorical_features].values
X_numeric = champions_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(champions_data['Winner'])
num_classes = len(label_encoder.classes_)

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)
X_test = X_test / np.max(X_test, axis=0)

# Convertir etiquetas a one-hot encoding
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Definir modelo de red neuronal con más complejidad
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

# Compilar modelo con hiperparámetros ajustados
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar modelo
model.fit(X_train, y_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.0417 - loss: 3.3553 - val_accuracy: 0.0000e+00 - val_loss: 3.3859
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0500 - loss: 3.3402 - val_accuracy: 0.0000e+00 - val_loss: 3.3862
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0250 - loss: 3.3411 - val_accuracy: 0.0000e+00 - val_loss: 3.3871
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0583 - loss: 3.3313 - val_accuracy: 0.0000e+00 - val_loss: 3.3875
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0333 - loss: 3.3436 - val_accuracy: 0.0000e+00 - val_loss: 3.3881
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0750 - loss: 3.3339 - val_accuracy: 0.0000e+00 - val_loss: 3.3889
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0417 - loss: 3.3232 - val_accuracy: 0.0000e+00 - val_loss: 3.3902
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0583 - loss: 3.3338 - val_accuracy: 0

Vemos que el entrenamiento inicial ha ido sin problemas. Vamos a probar a ver que predicciones realizaría nuestra red neuronal en base a los partidos de la Champions utilizada para obtener una medida de precisión real. Para ello primero vamos a medir la precisión de una predicción en general.

In [343]:
# Hacer predicciones en los datos de prueba
predictions = model.predict(X_test)

# Convertir las predicciones de vuelta a etiquetas
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

# Calcular la precisión
accuracy = np.mean(predicted_labels == true_labels)
print("Accuracy on test data:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Accuracy on test data: 0.04


Como puede verse con este resultado y la medición general anterior, la precisión del modelo es casi inexistente, por lo que vamos a seguir entrenándolo. Primero vamos a aplicar ingeniería de características para darle al modelo más variables sobre las que extraer información:

In [344]:
# Calcular la diferencia de goles entre los equipos
champions_data['Goal Difference'] = champions_data['Home Team Goals'] - champions_data['Away Team Goals']

# Calcular la proporción de goles marcados por cada equipo en relación con el total de goles en el partido
champions_data['Home Goals Ratio'] = champions_data['Home Team Goals'] / champions_data['Match Goals']
champions_data['Away Goals Ratio'] = champions_data['Away Team Goals'] / champions_data['Match Goals']

# Ver las primeras filas de los datos
print(champions_data.head())


   Match Number Round Number  Home Team  Away Team Result  Winner  \
0             1            1         20         61  1 - 2      61   
1             2            1         11         26  3 - 0      11   
2            13            1          2         52  3 - 0       2   
3            14            1         50          8  0 - 5       8   
4            25            1          9         66  6 - 0       9   

   Home Team Goals  Away Team Goals  Match Goals  Goal Difference  \
0                1                2            3               -1   
1                3                0            3                3   
2                3                0            3                3   
3                0                5            5               -5   
4                6                0            6                6   

   Home Goals Ratio  Away Goals Ratio  
0          0.333333          0.666667  
1          1.000000          0.000000  
2          1.000000          0.000000  
3         

In [345]:
# Ver las estadísticas de los datos
print(champions_data.describe())

       Match Number   Home Team   Away Team     Winner  Home Team Goals  \
count    125.000000  125.000000  125.000000  125.00000       125.000000   
mean      63.000000   25.288000   25.320000   17.24800         1.784000   
std       36.228442   28.268247   28.242784   22.06722         1.532361   
min        1.000000    1.000000    1.000000    1.00000         0.000000   
25%       32.000000    6.000000    6.000000    4.00000         1.000000   
50%       63.000000   14.000000   14.000000   10.00000         1.000000   
75%       94.000000   36.000000   36.000000   23.00000         3.000000   
max      125.000000  119.000000  119.000000  119.00000         7.000000   

       Away Team Goals  Match Goals  Goal Difference  Home Goals Ratio  \
count       125.000000   125.000000       125.000000        118.000000   
mean          1.424000     3.208000         0.360000          0.558787   
std           1.398525     1.622901         2.444216          0.344192   
min           0.000000     0

In [346]:
# Ver los datos nulos
print(champions_data.isnull().sum())

Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    7
Away Goals Ratio    7
dtype: int64


In [347]:
# Ver las columnas Home Goals Ratio y Away Goals Ratio donde hay datos nulos
print(champions_data[['Home Goals Ratio', 'Away Goals Ratio']], '\n')
print(champions_data[champions_data['Home Goals Ratio'].isnull()])

     Home Goals Ratio  Away Goals Ratio
0            0.333333          0.666667
1            1.000000          0.000000
2            1.000000          0.000000
3            0.000000          1.000000
4            1.000000          0.000000
..                ...               ...
120          0.714286          0.285714
121          0.333333          0.666667
122          0.500000          0.500000
123          0.666667          0.333333
124          0.750000          0.250000

[125 rows x 2 columns] 

     Match Number          Round Number  Home Team  Away Team Result  Winner  \
5              26                     1         14          7  0 - 0       7   
40             29                     3         66          7  0 - 0       7   
55             44                     4         40          3  0 - 0       3   
78             45                     5          6          3  0 - 0       3   
102           103  Round of 16 - Game 1         10         11  0 - 0      11   
104           

Vemos aquí el problema de partidos donde no hay goles. De momento vamos a dejarlo como está para darle más realismo al entrenamiento de la red neuronal:

In [348]:
# Preprocesamiento de datos con nuevas características
X = champions_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']].values
y = champions_data['Winner'].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']

X_categorical = champions_data[categorical_features].values
X_numeric = champions_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(champions_data['Winner'])
num_classes = len(label_encoder.classes_)
y = to_categorical(y, num_classes=num_classes)

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)
X_test = X_test / np.max(X_test, axis=0)


Ahora vamos a ampliar el alcance de nuestra red neuronal:

In [349]:
# Definir modelo de red neuronal con arquitectura modificada
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Cambiar a 'softmax' y ajustar neuronas

# Compilar y entrenar el modelo...
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=40, validation_split=0.3)

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.0548 - loss: 3.3671 - val_accuracy: 0.0667 - val_loss: 3.3668
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0726 - loss: 3.3662 - val_accuracy: 0.0667 - val_loss: 3.3663
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0810 - loss: 3.3653 - val_accuracy: 0.0667 - val_loss: 3.3659
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.0833 - loss: 3.3644 - val_accuracy: 0.0000e+00 - val_loss: 3.3654
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.0917 - loss: 3.3634 - val_accuracy: 0.0000e+00 - val_loss: 3.3649
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0548 - loss: 3.3625 - val_accuracy: 0.1333 - val_loss: 3.3644
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0643 - loss: 3.3618 - val_accuracy: 0.1333 - val_loss: 3.3640
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0893 - loss: 3.3611 - val_accuracy: 0.1333 - val_loss: 3.

Vamos a continuar añadiéndo datos de entrenamiento. Para ello vamos a prepararlos primero:

In [350]:
champions_data2 = pd.read_csv("data/champions-league-2018.csv")
champions_data3 = pd.read_csv("data/champions-league-2019.csv")
champions_data4 = pd.read_csv("data/champions-league-2020.csv")
champions_data5 = pd.read_csv("data/champions-league-2021.csv")
champions_data6 = pd.read_csv("data/champions-league-2022.csv")

# Reemplazar nombres de equipos por números en las columnas correspondientes
champions = [champions_data2, champions_data3, champions_data4, champions_data5, champions_data6]
for data in champions:
    data['Home Team'] = data['Home Team'].map(team_mapping)
    data['Away Team'] = data['Away Team'].map(team_mapping)
    data['Winner'] = data['Winner'].map(team_mapping)
    # Calcular la diferencia de goles entre los equipos
    data['Goal Difference'] = data['Home Team Goals'] - data['Away Team Goals']

    # Calcular la proporción de goles marcados por cada equipo en relación con el total de goles en el partido
    data['Home Goals Ratio'] = data['Home Team Goals'] / data['Match Goals']
    data['Away Goals Ratio'] = data['Away Team Goals'] / data['Match Goals']

In [351]:
# Ver los datos nulos de cada conjunto de datos
for i, data in enumerate(champions):
    print(f"Data {i + 2} null values:\n{data.isnull().sum()}\n")

Data 2 null values:
Match Number        0
Round Number        0
Home Team           3
Away Team           3
Result              0
Winner              1
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    7
Away Goals Ratio    7
dtype: int64

Data 3 null values:
Match Number        0
Round Number        0
Home Team           3
Away Team           3
Result              0
Winner              1
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    4
Away Goals Ratio    4
dtype: int64

Data 4 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    6
Away Goals Ratio    6
dtype: int64

Data 5 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           

Vemos que el csv 2 y 3 tienen datos nulos en partidos. Vamos a visualizarlos:

In [352]:
#Ver las filas Home Team, Away Team y Winner donde hay datos nulos en champions_data2
# Seleccionar las filas donde hay valores nulos en las columnas específicas
null_data = champions_data2[champions_data2[['Home Team', 'Away Team', 'Winner']].isnull().any(axis=1)]

# Mostrar las filas seleccionadas
print(null_data[['Home Team', 'Away Team', 'Winner']])

    Home Team  Away Team  Winner
4         NaN       16.0    16.0
24        8.0        NaN     8.0
44        5.0        NaN     5.0
49        NaN        5.0     NaN
77       16.0        NaN    16.0
86        NaN        8.0     8.0


In [353]:
#Ver las filas Home Team, Away Team y Winner donde hay datos nulos en champions_data3
# Seleccionar las filas donde hay valores nulos en las columnas específicas
null_data = champions_data3[champions_data3[['Home Team', 'Away Team', 'Winner']].isnull().any(axis=1)]

# Mostrar las filas seleccionadas
print(null_data[['Home Team', 'Away Team', 'Winner']])

    Home Team  Away Team  Winner
11        2.0        NaN     2.0
20        NaN       40.0     NaN
37       23.0        NaN    23.0
60        NaN       23.0    23.0
67        NaN        2.0     2.0
93       40.0        NaN    40.0


In [354]:
# Concatenar todos los datos verticalmente
combined_data = pd.concat([champions_data2, champions_data3, champions_data4, champions_data5, champions_data6], axis=0, ignore_index=True)

Vamos a analizar el resultado de la unión:

In [355]:
# Ver las filas union de los datos
print(combined_data.head())

   Match Number Round Number  Home Team  Away Team Result  Winner  \
0             1            1       18.0       23.0  2 - 1    18.0   
1             2            1        3.0       32.0  4 - 0     3.0   
2             3            1       36.0        7.0  1 - 2     7.0   
3             4            1       31.0       13.0  0 - 1    13.0   
4             5            1        NaN       16.0  0 - 0    16.0   

   Home Team Goals  Away Team Goals  Match Goals  Goal Difference  \
0                2                1            3                1   
1                4                0            4                4   
2                1                2            3               -1   
3                0                1            1               -1   
4                0                0            0                0   

   Home Goals Ratio  Away Goals Ratio  
0          0.666667          0.333333  
1          1.000000          0.000000  
2          0.333333          0.666667  
3         

In [356]:
# Ver los datos
print(combined_data.describe())

       Match Number   Home Team   Away Team      Winner  Home Team Goals  \
count    619.000000  613.000000  613.000000  617.000000       619.000000   
mean      62.445880   33.960848   33.081566   24.763371         1.621971   
std       35.841997   70.837717   68.288243   64.087722         1.412806   
min        1.000000    1.000000    1.000000    1.000000         0.000000   
25%       31.500000    7.000000    7.000000    5.000000         1.000000   
50%       62.000000   18.000000   18.000000   11.000000         1.000000   
75%       93.000000   31.000000   31.000000   23.000000         2.000000   
max      125.000000  502.000000  502.000000  502.000000         7.000000   

       Away Team Goals  Match Goals  Goal Difference  Home Goals Ratio  \
count       619.000000   619.000000       619.000000        586.000000   
mean          1.399031     3.021002         0.222940          0.545596   
std           1.384402     1.742635         2.188242          0.364635   
min           0.000

In [357]:
# Ver los datos nulos
print(combined_data.isnull().sum())

Match Number         0
Round Number         0
Home Team            6
Away Team            6
Result               0
Winner               2
Home Team Goals      0
Away Team Goals      0
Match Goals          0
Goal Difference      0
Home Goals Ratio    33
Away Goals Ratio    33
dtype: int64


In [358]:
#mostrar las filas con datos nulos
print(combined_data[combined_data.isnull().any(axis=1)])

     Match Number Round Number  Home Team  Away Team Result  Winner  \
4               5            1        NaN       16.0  0 - 0    16.0   
23             24            2       11.0       61.0  0 - 0    61.0   
24             25            2        8.0        NaN  6 - 1     8.0   
44             45            3        5.0        NaN  4 - 0     5.0   
46             47            3       64.0       51.0  0 - 0    51.0   
49             50            4        NaN        5.0  2 - 0     NaN   
74             75            5       13.0       31.0  0 - 0    31.0   
77             78            5       16.0        NaN  3 - 1    16.0   
83             84            6       31.0        7.0  0 - 0     7.0   
86             87            6        NaN        8.0  1 - 4     8.0   
100           101   R16 Game 1       27.0        3.0  0 - 0     3.0   
101           102   R16 Game 1        5.0        2.0  0 - 0     2.0   
129             5            1       13.0        3.0  0 - 0     3.0   
133   

In [359]:
# Preprocesamiento de datos
X = combined_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']].values
y = combined_data['Winner'].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']

X_categorical = combined_data[categorical_features].values
X_numeric = combined_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(combined_data['Winner'])
num_classes = len(label_encoder.classes_)

# Convertir etiquetas a one-hot encoding
y = to_categorical(y, num_classes=num_classes)

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)

# Definir modelo de red neuronal con más complejidad
model.fit(X_train, y_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "C:\Users\javie\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\javie\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\javie\AppData\Local\Programs\Python\Python310\lib\asyncio\base_ev